
## 🧪 Gray–Scott Reaction–Diffusion Field

This section generates a **2D Gray–Scott reaction–diffusion field** used to simulate natural folding and growth patterns (similar to coral, brain tissue, or surface marbling).  
The resulting field will later be applied as a **displacement map** to deform a mesh surface.

**Key parameters**
- `Du`, `Dv`: Diffusion rates of two reactants.
- `F`, `k`: Feed and kill rates controlling pattern density and wavelength.
- `n_seeds`: Number of initial reaction centres (intersecting fronts create complex folds).
- `steps`: Number of simulation iterations — more steps produce more stable, detailed fields.

Run this code block first to define `gray_scott_multi()` and generate a field array for later use.

In [ ]:
import numpy as np
import trimesh

def gray_scott_multi(size=256, steps=20000, Du=0.16, Dv=0.08, F=0.060, k=0.062, n_seeds=7):
    U = np.ones((size, size))
    V = np.zeros((size, size))
    rng = np.random.default_rng(42)

    for _ in range(n_seeds):
        cx, cy = rng.integers(size//4, 3*size//4, 2)
        r = size // 12
        U[cx-r:cx+r, cy-r:cy+r] = 0.5
        V[cx-r:cx+r, cy-r:cy+r] = 0.25

    for _ in range(steps):
        lapU = np.roll(U,1,0)+np.roll(U,-1,0)+np.roll(U,1,1)+np.roll(U,-1,1)-4*U
        lapV = np.roll(V,1,0)+np.roll(V,-1,0)+np.roll(V,1,1)+np.roll(V,-1,1)-4*V
        UVV = U*V*V
        U += Du*lapU - UVV + F*(1-U)
        V += Dv*lapV + UVV - (F+k)*V
    return V

def rounded_square_path(side=100, radius=20, n_per_edge=64):
    """Return x,y points of a rounded square loop."""
    half = side / 2 - radius
    pts = []
    # bottom edge → right corner
    for t in np.linspace(0, 1, n_per_edge):
        pts.append([-half + t*2*half, -half - radius])
    for a in np.linspace(-np.pi/2, 0, n_per_edge//4):
        pts.append([half + radius*np.cos(a), -half + radius*np.sin(a)])
    # right edge → top corner
    for t in np.linspace(0, 1, n_per_edge):
        pts.append([half + radius, -half + t*2*half])
    for a in np.linspace(0, np.pi/2, n_per_edge//4):
        pts.append([half + radius*np.cos(a), half + radius*np.sin(a)])
    # top edge → left corner
    for t in np.linspace(0, 1, n_per_edge):
        pts.append([half - t*2*half, half + radius])
    for a in np.linspace(np.pi/2, np.pi, n_per_edge//4):
        pts.append([-half + radius*np.cos(a), half + radius*np.sin(a)])
    # left edge → bottom corner
    for t in np.linspace(0, 1, n_per_edge):
        pts.append([-half - radius, half - t*2*half])
    for a in np.linspace(np.pi, 3*np.pi/2, n_per_edge//4):
        pts.append([-half + radius*np.cos(a), -half + radius*np.sin(a)])
    return np.array(pts)

def make_square_column_field(field, height=200, side=100, corner_radius=20, amplitude=8):
    h, w = field.shape
    path = rounded_square_path(side, corner_radius, n_per_edge=w//4)
    z_vals = np.linspace(0, height, h)

    # vertical taper mask
    taper = np.ones(h)
    top_taper_start = int(h * 2/3)
    bottom_taper_end = int(h * 1/6)
    for i in range(h):
        if i < bottom_taper_end:
            taper[i] = i / bottom_taper_end
        elif i > top_taper_start:
            taper[i] = 1 - (i - top_taper_start) / (h - top_taper_start)

    verts = []
    for i, zi in enumerate(z_vals):
        disp = amplitude * taper[i] * (field[i, :] - 0.5)
        layer = np.column_stack([
            path[:, 0] * (1 + disp / side),
            path[:, 1] * (1 + disp / side),
            np.full_like(path[:, 0], zi)
        ])
        verts.append(layer)
    verts = np.vstack(verts)

    faces = []
    p = len(path)
    for i in range(h-1):
        for j in range(p-1):
            a = i*p + j
            b = a + 1
            c = a + p
            d = c + 1
            faces.append([a,b,c])
            faces.append([b,d,c])
        # close loop
        a = i*p + (p-1)
        b = i*p
        c = a + p
        d = b + p
        faces.append([a,b,c])
        faces.append([b,d,c])

    return trimesh.Trimesh(vertices=verts, faces=faces)

# --- run
field = gray_scott_multi(size=256, steps=20000, n_seeds=7)
mesh = make_square_column_field(field, height=200, side=100, corner_radius=20, amplitude=8)
mesh.export("brain_squarecolumn_tapered.stl")


## 🧊 Applying the Reaction–Diffusion Field to an Existing STL Mesh

This section loads an existing `.stl` 3D model, computes vertex normals, and displaces each vertex along its normal using values derived from the **Gray–Scott field**.  
The resulting surface inherits the model’s geometry while gaining **organic folds**.

**Pipeline overview**
1. Load mesh using `trimesh`.
2. Compute vertex normals for deformation direction.
3. Sample or interpolate reaction–diffusion field values across vertices.
4. Apply vertical tapering (reduce folds near top and bottom).
5. Displace vertices along normals proportionally to field intensity.
6. Export the deformed result as a new `.stl`.

> ⚠️ The input mesh should be manifold and reasonably smooth.  
> Non-uniform scaling or inverted normals can cause visible artefacts in the displacement.


In [ ]:
import trimesh
import numpy as np

# load mesh
mesh = trimesh.load('input_model.stl')

# normals
normals = mesh.vertex_normals

# gray-scott field
field = gray_scott_multi(size=256, steps=20000, n_seeds=7)
field_interp = np.random.choice(field.flatten(), len(mesh.vertices))

# vertical taper by Z height
z = mesh.vertices[:,2]
z_norm = (z - z.min()) / (z.max() - z.min())
taper = np.clip(np.interp(z_norm, [0, 0.15, 0.65, 1.0], [0, 1, 1, 0]), 0, 1)

# apply displacement
disp = 8 * taper * (field_interp - 0.5)
mesh.vertices += normals * disp[:, None]

mesh.export('deformed_model.stl')
